In [ ]:
import numpy as np
import pandas as pd
from sklearn import preprocessing
try:
  # %tensorflow_version only exists in Colab.
  %tensorflow_version 2.x
except Exception:
  pass
import tensorflow as tf
import numpy as np
import matplotlib.pyplot as plt
from sklearn.model_selection import train_test_split, cross_val_score, KFold
from PIL import Image
from tensorflow import keras
from tensorflow.keras import datasets,layers,optimizers,Sequential,metrics

In [ ]:
def save_img(imgs,names):
  img_new = Image.new('L',(280,280))
  index = 0
  for i in range(0,280,80):
    for j in range(0,280,80):
      img = imgs[index]
      img = Image.fromarray(img,mode='L')
      img_new.paste(img,(i,j))
      index+=1
  img_new.save(names)

In [ ]:
def feature_scale(x):
  x = tf.cast(x,dtype=tf.float32)/255.
  return x

In [ ]:
#Dim reduct nums
dim_reduce = 10
batch_num = 128
lr = 1e-3

In [ ]:
(x,y),(x_test,y_test) = datasets.fashion_mnist.load_data()
data = tf.data.Dataset.from_tensor_slices(x)
data = data.map(feature_scale).shuffle(10000).batch(batch_num)

data_test = tf.data.Dataset.from_tensor_slices(x_test)
data_test = data_test.map(feature_scale).batch(batch_num)
data_iter = iter(data)
samples = next(data_iter)
print(samples[0].shape,samples[1].shape)

4423680/4422102 [==============================] - 0s 0us/step
(28, 28) (28, 28)


In [ ]:
class AE(keras.Model):
  def __init__(self):
    super(AE,self).__init__()
    #encoder
    self.model_encoder = Sequential([layers.Dense(256,activation=tf.nn.relu),
                             layers.Dense(128,activation=tf.nn.relu),
                             layers.Dense(dim_reduce,activation=tf.nn.relu),
                                     
                            ])
    
    #decoder
    self.model_decoder = Sequential([layers.Dense(128,activation=tf.nn.relu),
                             layers.Dense(256,activation=tf.nn.relu),
                             layers.Dense(784,activation=tf.nn.relu),
                                     
                            ])

  def call(self, inputs, training=None):
    x = self.model_encoder(inputs)
    x = self.model_decoder(x)
    return x


In [ ]:
model = AE()
model.build(input_shape=(None,784))
model.summary()

Model: "ae"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
sequential (Sequential)      multiple                  235146    
_________________________________________________________________
sequential_1 (Sequential)    multiple                  235920    
Total params: 471,066
Trainable params: 471,066
Non-trainable params: 0
_________________________________________________________________


In [ ]:
!rm -rf img_result
!mkdir img_result

In [ ]:
optimizer = optimizers.Adam(lr=lr)
for i in range(10):
  for step,x in enumerate(data):
    x = tf.reshape(x,[-1,784])
    with tf.GradientTape() as tape:
      logits = model(x)
      loss = tf.losses.binary_crossentropy(x,logits,from_logits=True)
      loss = tf.reduce_mean(loss)
    grads = tape.gradient(loss,model.trainable_variables)
    optimizer.apply_gradients(zip(grads,model.trainable_variables))
    
    if step %100==0:
      print(i,step,'loss:',float(loss))
      
  x = next(iter(data_test))
  val_x = tf.reshape(x,[-1,784])
  logits = model(val_x)
  x_hat = tf.sigmoid(logits)
  x_hat = tf.reshape(x_hat,[-1,28,28])
  x_hat = x_hat.numpy()*255
  x_hat = x_hat.astype(np.uint8)
  save_img(x_hat,'img_result/AE_img_%d.png'%i)

0 0 loss: 0.6942018866539001
0 100 loss: 0.6850414276123047
0 200 loss: 0.6786310076713562
0 300 loss: 0.6799616813659668
0 400 loss: 0.6795993447303772
1 0 loss: 0.679826021194458
1 100 loss: 0.6794265508651733
1 200 loss: 0.6824880838394165
1 300 loss: 0.6780180335044861
1 400 loss: 0.6764596104621887
2 0 loss: 0.6795350313186646
2 100 loss: 0.6776262521743774
2 200 loss: 0.6781857013702393
2 300 loss: 0.6745344400405884
2 400 loss: 0.6731181144714355
3 0 loss: 0.672815203666687
3 100 loss: 0.6690542697906494
3 200 loss: 0.6690125465393066
3 300 loss: 0.6686781644821167
3 400 loss: 0.6675901412963867
4 0 loss: 0.6702392101287842
4 100 loss: 0.6623368859291077
4 200 loss: 0.6615491509437561
4 300 loss: 0.6631669998168945
4 400 loss: 0.663107693195343
5 0 loss: 0.6644995212554932
5 100 loss: 0.6664038896560669
5 200 loss: 0.6646358966827393
5 300 loss: 0.6668475866317749
5 400 loss: 0.6610640287399292
6 0 loss: 0.6575562953948975
6 100 loss: 0.6653007864952087
6 200 loss: 0.65699720382

In [ ]:
from google.colab import files
files.download('img_result/AE_img_9.png')

In [ ]:
x = next(iter(data_test))
val_x = tf.reshape(x,[-1,784])
logits = model(val_x)
x_pred = tf.sigmoid(logits)
x_pred = tf.reshape(x_pred,[-1,28,28])
x_pred = tf.concat([x,x_pred],axis=0)
x_pred = x_pred.numpy()*255
x_pred = x_pred.astype(np.uint8)
save_img(x_pred,'img_result/AE_img_total.png')